<a href="https://colab.research.google.com/github/BogdanGFTP/digit_predictions/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive, files
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, CSVLogger

TRAINING_LOGS_FILE = "training_logs.csv"
MODEL_FILE = "model.h5"
KAGGLE_SUBMISSION_FILE = "kaggle_submission.csv"

In [0]:
VERBOSITY = 1
EPOCHS = 100
BATCH_SIZE = 512
CLASSES = 10
CHANNELS = 1
IMAGE_SIZE = 28
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
VALIDATION_RATIO = 0.1

In [0]:
path = "drive/My Drive/MNIST/"
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

y = train["label"]
x = train.drop(labels = ["label"], axis = 1) 

# Reshape data
x = x.values.reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)
test = test.values.reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)

# One-Hot encoding
y = to_categorical(y, num_classes=CLASSES)

In [16]:
import gc; gc.enable()
gc.collect()

7

In [0]:
# Prepare training/validation sets
x_training, x_validation, y_training, y_validation = train_test_split(x,
                                                                      y,
                                                                      test_size=VALIDATION_RATIO,
                                                                      shuffle = True)


In [23]:
inp = Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
x = Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation='relu')(inp)
x = MaxPool2D(pool_size=(2,2))(x)
x = Dropout(0.5)(x)
x = Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu')(x)
x = Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu')(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(8192, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(8192, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(CLASSES, activation="softmax")(x)

model = Model(inp, output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [0]:
model.compile(optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.00001), loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
# Data augmentation
data_generator = ImageDataGenerator(rescale=1./255,
                                    rotation_range=1,
                                    zoom_range=0.1, 
                                    width_shift_range=0.05,
                                    height_shift_range=0.05)
data_generator.fit(x_training)

In [37]:
gc.collect()

156

In [29]:
# Training
history = model.fit_generator(data_generator.flow(x_training,
                                                  y_training,
                                                  batch_size=BATCH_SIZE),
                              epochs=EPOCHS,
                              validation_data=(x_validation, y_validation),
                              verbose=VERBOSITY,
                              steps_per_epoch=x_training.shape[0] // BATCH_SIZE,
                              callbacks=[CSVLogger(TRAINING_LOGS_FILE,
                                                   append=False,
                                                   separator=";")])
model.save_weights(MODEL_FILE)

Epoch 1/100
73/73 [==============================] - 23s 314ms/step - loss: 1.1336 - acc: 0.6198 - val_loss: 1.7201 - val_acc: 0.8902
Epoch 2/100
73/73 [==============================] - 18s 240ms/step - loss: 0.4965 - acc: 0.8378 - val_loss: 0.8771 - val_acc: 0.9450
Epoch 3/100
73/73 [==============================] - 17s 238ms/step - loss: 0.3200 - acc: 0.8955 - val_loss: 0.6787 - val_acc: 0.9569
Epoch 4/100
73/73 [==============================] - 17s 238ms/step - loss: 0.2245 - acc: 0.9275 - val_loss: 0.5300 - val_acc: 0.9664
Epoch 5/100
73/73 [==============================] - 17s 238ms/step - loss: 0.1837 - acc: 0.9414 - val_loss: 0.6064 - val_acc: 0.9624
Epoch 6/100
73/73 [==============================] - 17s 238ms/step - loss: 0.1504 - acc: 0.9518 - val_loss: 0.3607 - val_acc: 0.9776
Epoch 7/100
73/73 [==============================] - 17s 238ms/step - loss: 0.1357 - acc: 0.9577 - val_loss: 0.3208 - val_acc: 0.9798
Epoch 8/100
73/73 [==============================] - 17s 238ms

In [38]:
# Testing
predictions = model.predict(test, verbose=1)
results = np.argmax(predictions,axis = 1)
pd.DataFrame({"ImageId":list(range(1,len(results)+1)),
              "Label":results}).to_csv(KAGGLE_SUBMISSION_FILE,
                                           index=False,
                                           header=True)

28000/28000 [==============================] - 9s 315us/step
